# Signal Processing in OCT

Raw OCT data requires significant processing to produce high-quality images. This notebook covers the essential signal processing techniques.

## Learning Objectives

By the end of this notebook, you will understand:
1. Spectral shaping and resampling
2. Dispersion compensation
3. Noise reduction techniques
4. Image enhancement methods
5. Complete OCT processing pipeline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft, fftfreq
from scipy.signal import butter, filtfilt, medfilt
from scipy.ndimage import gaussian_filter, median_filter

plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("Libraries loaded successfully!")

## 1. Spectral Resampling

For proper FFT, data must be evenly spaced in **k-space** (wavenumber), not wavelength.

$$k = \frac{2\pi}{\lambda}$$

Most spectrometers measure evenly in $\lambda$, so we need to resample to linear k.

In [ ]:
def resample_to_k_space(wavelengths, spectrum):
    """
    Resample spectrum from linear wavelength to linear k-space.
    """
    # Calculate wavenumbers
    k = 2 * np.pi / wavelengths
    
    # Create linear k-space grid
    k_linear = np.linspace(k.min(), k.max(), len(k))
    
    # Interpolate (note: k is reversed from wavelength)
    spectrum_k = np.interp(k_linear, k[::-1], spectrum[::-1])
    
    return k_linear, spectrum_k

# Example: Show effect of resampling
wavelengths = np.linspace(800e-9, 880e-9, 1024)
lambda0 = 840e-9
spectrum = np.exp(-((wavelengths - lambda0) / 20e-9)**2)

# Add modulation (representing depth structure)
depth = 100e-6
k_orig = 2 * np.pi / wavelengths
spectrum = spectrum * (1 + 0.5 * np.cos(2 * k_orig * depth))

# Resample
k_linear, spectrum_k = resample_to_k_space(wavelengths, spectrum)

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Lambda space
axes[0].plot(wavelengths * 1e9, spectrum, 'b', linewidth=2)
axes[0].set_xlabel('Wavelength (nm)', fontsize=12)
axes[0].set_ylabel('Intensity', fontsize=12)
axes[0].set_title('Original Spectrum (λ-space)', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# K space
axes[1].plot(k_linear, spectrum_k, 'r', linewidth=2)
axes[1].set_xlabel('Wavenumber k (rad/m)', fontsize=12)
axes[1].set_ylabel('Intensity', fontsize=12)
axes[1].set_title('Resampled Spectrum (k-space)', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Resampling to k-space is essential for accurate FFT!")

## 2. Dispersion Compensation

**Dispersion** occurs when different wavelengths travel at different speeds through optical elements.

### Effects:
- Broadens the axial PSF
- Reduces resolution
- Decreases signal strength

### Compensation:
Apply a phase correction in the spectral domain:
$$S_{corrected}(k) = S(k) \cdot e^{-i\phi(k)}$$

where $\phi(k) = a_2 k^2 + a_3 k^3 + ...$

In [ ]:
def add_dispersion(k, spectrum, a2=1e-25, a3=1e-39):
    """
    Add dispersion to spectrum (simulate unmatched optical path).
    """
    k0 = np.mean(k)
    dk = k - k0
    
    # Dispersion phase
    phase = a2 * dk**2 + a3 * dk**3
    
    # Apply to spectrum
    dispersed = spectrum * np.exp(1j * phase)
    
    return dispersed

def compensate_dispersion(k, spectrum_complex, a2=0, a3=0):
    """
    Compensate dispersion by applying opposite phase.
    """
    k0 = np.mean(k)
    dk = k - k0
    
    # Compensation phase (opposite sign)
    phase = -(a2 * dk**2 + a3 * dk**3)
    
    # Apply compensation
    compensated = spectrum_complex * np.exp(1j * phase)
    
    return compensated

# Generate test signal
k_linear = np.linspace(7.1e6, 7.9e6, 2048)  # rad/m
wavelengths_test = 2 * np.pi / k_linear

# Source spectrum
lambda0 = np.mean(wavelengths_test)
spectrum_clean = np.exp(-((wavelengths_test - lambda0) / (40e-9))**2)

# Add single reflector
depth = 150e-6
spectrum_clean = spectrum_clean * (1 + 0.8 * np.cos(2 * k_linear * depth))

# Add dispersion
spectrum_dispersed = add_dispersion(k_linear, spectrum_clean, a2=2e-25, a3=5e-39)

# Process both
a_scan_clean = np.abs(fft(spectrum_clean))
a_scan_dispersed = np.abs(fft(spectrum_dispersed))

# Compensate
spectrum_compensated = compensate_dispersion(k_linear, spectrum_dispersed, a2=2e-25, a3=5e-39)
a_scan_compensated = np.abs(fft(spectrum_compensated))

# Depth axis
dk = k_linear[1] - k_linear[0]
depth_range = np.pi / dk
depths = np.linspace(0, depth_range, len(a_scan_clean)) * 1e6  # Convert to μm

# Plot comparison
plt.figure(figsize=(14, 6))

plt.plot(depths[:1024], a_scan_clean[:1024], 'g', linewidth=2, label='No dispersion', alpha=0.7)
plt.plot(depths[:1024], a_scan_dispersed[:1024], 'r', linewidth=2, label='With dispersion', alpha=0.7)
plt.plot(depths[:1024], a_scan_compensated[:1024], 'b--', linewidth=2, label='Compensated', alpha=0.7)

plt.xlabel('Depth (μm)', fontsize=12)
plt.ylabel('Intensity (a.u.)', fontsize=12)
plt.title('Effect of Dispersion Compensation', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.xlim(50, 250)
plt.tight_layout()
plt.show()

print("Notice: Dispersion broadens peaks and reduces amplitude")
print("Compensation restores the original sharp peak!")

## 3. Background Subtraction

Remove DC and fixed-pattern noise by subtracting a reference spectrum.

In [ ]:
def background_subtraction(spectrum, background=None):
    """
    Subtract background and remove DC component.
    """
    if background is not None:
        spectrum = spectrum - background
    
    # Remove DC (mean)
    spectrum = spectrum - np.mean(spectrum)
    
    return spectrum

# Example
print("Background subtraction removes:")
print("  ✓ DC offset from source spectrum")
print("  ✓ Fixed pattern noise")
print("  ✓ Auto-correlation artifact at zero depth")
print("\nImproves image quality significantly!")

## 4. Windowing (Apodization)

Apply a window function to reduce sidelobes in the PSF.

Common windows:
- **Hann**: Good balance
- **Hamming**: Slightly better sidelobe suppression
- **Gaussian**: Smooth, minimal ringing

In [ ]:
def apply_window(spectrum, window_type='hann'):
    """
    Apply window function to spectrum.
    """
    n = len(spectrum)
    
    if window_type == 'hann':
        window = np.hanning(n)
    elif window_type == 'hamming':
        window = np.hamming(n)
    elif window_type == 'gaussian':
        x = np.linspace(-3, 3, n)
        window = np.exp(-x**2)
    else:
        window = np.ones(n)  # Rectangular (no window)
    
    return spectrum * window

# Compare windows
k_test = np.linspace(7.1e6, 7.9e6, 2048)
spectrum_test = np.exp(-((2*np.pi/k_test - 840e-9) / (40e-9))**2)
spectrum_test = spectrum_test * (1 + 0.9 * np.cos(2 * k_test * 150e-6))

windows = ['none', 'hann', 'hamming', 'gaussian']
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for ax, window_type in zip(axes.flat, windows):
    windowed = apply_window(spectrum_test - np.mean(spectrum_test), window_type)
    a_scan = np.abs(fft(windowed))
    
    depths = np.linspace(0, np.pi/(k_test[1]-k_test[0]), len(a_scan)) * 1e6
    
    ax.plot(depths[:1024], 20*np.log10(a_scan[:1024]/np.max(a_scan)), linewidth=2)
    ax.set_xlabel('Depth (μm)')
    ax.set_ylabel('Intensity (dB)')
    ax.set_title(f'{window_type.capitalize()} Window', fontweight='bold')
    ax.set_ylim(-60, 0)
    ax.grid(True, alpha=0.3)
    ax.set_xlim(50, 250)

plt.tight_layout()
plt.show()

print("Windows reduce sidelobes but slightly broaden the main peak")
print("Trade-off: Resolution vs. sidelobe suppression")

## 5. Image Enhancement: Noise Reduction

Common techniques for B-scan enhancement:
1. **Median filtering**: Removes speckle
2. **Gaussian filtering**: Smooths image
3. **Averaging**: Multiple acquisitions

In [ ]:
# Generate noisy B-scan
np.random.seed(42)
bscan_size = (300, 200)
bscan_clean = np.zeros(bscan_size)

# Add some layer structures
for x in range(bscan_size[1]):
    layer1 = 50 + 10 * np.sin(2 * np.pi * x / 50)
    layer2 = 120 + 8 * np.sin(2 * np.pi * x / 70)
    layer3 = 200 + 6 * np.sin(2 * np.pi * x / 90)
    
    for layer, intensity in [(layer1, 0.8), (layer2, 0.6), (layer3, 0.4)]:
        y = int(layer)
        if 0 <= y < bscan_size[0]:
            for dy in range(-3, 4):
                if 0 <= y+dy < bscan_size[0]:
                    bscan_clean[y+dy, x] += intensity * np.exp(-(dy/2)**2)

# Add noise
noise = np.random.randn(*bscan_size) * 0.1
speckle = np.random.rayleigh(0.05, bscan_size)
bscan_noisy = bscan_clean + noise + speckle

# Apply filters
bscan_median = median_filter(bscan_noisy, size=3)
bscan_gaussian = gaussian_filter(bscan_noisy, sigma=1.0)

# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].imshow(bscan_clean, cmap='gray', aspect='auto')
axes[0, 0].set_title('Ground Truth', fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(bscan_noisy, cmap='gray', aspect='auto')
axes[0, 1].set_title('Noisy Image', fontweight='bold')
axes[0, 1].axis('off')

axes[1, 0].imshow(bscan_median, cmap='gray', aspect='auto')
axes[1, 0].set_title('Median Filtered', fontweight='bold')
axes[1, 0].axis('off')

axes[1, 1].imshow(bscan_gaussian, cmap='gray', aspect='auto')
axes[1, 1].set_title('Gaussian Filtered', fontweight='bold')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("Filtering reduces noise but may blur fine structures")
print("Choose filter type based on application requirements")

## 6. Complete Processing Pipeline

Put it all together!

In [ ]:
def oct_processing_pipeline(wavelengths, spectrum, 
                             dispersion_params=None,
                             window_type='hann',
                             log_scale=True):
    """
    Complete OCT signal processing pipeline.
    
    Steps:
    1. Background subtraction
    2. Resampling to k-space
    3. Dispersion compensation
    4. Windowing
    5. FFT
    6. Magnitude calculation
    7. Log scaling
    """
    # 1. Background subtraction
    spectrum = spectrum - np.mean(spectrum)
    
    # 2. Resample to k-space
    k = 2 * np.pi / wavelengths
    k_linear = np.linspace(k.min(), k.max(), len(k))
    spectrum_k = np.interp(k_linear, k[::-1], spectrum[::-1])
    
    # 3. Dispersion compensation
    if dispersion_params is not None:
        k0 = np.mean(k_linear)
        dk = k_linear - k0
        a2, a3 = dispersion_params
        phase = -(a2 * dk**2 + a3 * dk**3)
        spectrum_k = spectrum_k * np.exp(1j * phase)
    
    # 4. Windowing
    spectrum_k = apply_window(spectrum_k, window_type)
    
    # 5. FFT
    a_scan_complex = fft(spectrum_k)
    
    # 6. Magnitude
    a_scan = np.abs(a_scan_complex)
    
    # 7. Log scale
    if log_scale:
        a_scan = 20 * np.log10(a_scan / np.max(a_scan))
        a_scan = np.clip(a_scan, -60, 0)
    
    # Depth axis
    dk = k_linear[1] - k_linear[0]
    depth_max = np.pi / dk
    depths = np.linspace(0, depth_max, len(a_scan))
    
    return depths, a_scan

print("Complete OCT Processing Pipeline:")
print("  1. Background Subtraction")
print("  2. K-space Resampling")
print("  3. Dispersion Compensation")
print("  4. Spectral Windowing")
print("  5. Fourier Transform")
print("  6. Magnitude Calculation")
print("  7. Logarithmic Scaling")
print("\nThis pipeline is essential for high-quality OCT imaging!")

## Summary

In this notebook, we've learned:

1. ✅ **K-space resampling**: Essential for accurate FFT

2. ✅ **Dispersion compensation**: Restores axial resolution

3. ✅ **Background subtraction**: Removes DC artifacts

4. ✅ **Windowing**: Reduces sidelobes

5. ✅ **Noise reduction**: Improves image quality

6. ✅ **Complete pipeline**: All steps integrated

## Processing Best Practices

1. **Always resample to k-space** before FFT
2. **Calibrate dispersion** for your specific system
3. **Choose window** based on resolution vs. sidelobe trade-off
4. **Apply filtering** judiciously to preserve features
5. **Use dB scale** for display (matches human perception)

## Advanced Topics (Not Covered)

- Phase-sensitive OCT
- Doppler OCT
- Polarization-sensitive OCT
- GPU acceleration
- Real-time processing

## Next Steps

In the final notebook, we'll explore:
- Clinical applications
- Case studies
- Current research directions
- Future of OCT technology

---

**Continue to Notebook 7: Applications and Examples →**